In [1]:
import logging
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML

In [3]:
logging.basicConfig(format='%(asctime)s %(levelname)s %(name)s %(message)s', level=logging.INFO)

In [22]:
import downloader

In [23]:
import download_datasets

In [24]:
dfs=download_datasets.get_datasets()

2022-07-23 18:20:48,104 INFO root processing: data_sources/MaunaLoaCO2MonthlyMean.toml
2022-07-23 18:20:51,328 INFO Downloader Download from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv: OK.
2022-07-23 18:20:51,343 INFO root processing: data_sources/EuropeanMeanTemperatureSinceRomanTime_EuroMed2k.toml
2022-07-23 18:20:52,839 INFO Downloader Download from https://www.ncei.noaa.gov/pub/data/paleo/pages2k/EuroMed2k/eujja_2krecon_nested_cps.txt: OK.
2022-07-23 18:20:52,860 INFO root processing: data_sources/MaunaLoaCO2WeeklyMeanHistorical.toml
2022-07-23 18:20:54,278 INFO Downloader Download from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_weekly_mlo.csv: OK.
2022-07-23 18:20:54,309 INFO root processing: data_sources/WikipediaGeologicTimeScales.toml
2022-07-23 18:20:54,841 INFO Downloader Download from https://en.wikipedia.org/wiki/Geologic_time_scale: OK.
2022-07-23 18:20:54,966 INFO root processing: data_sources/DWDGermanJuneMeanTemperature.toml
2022-07-23 18:20:55,14

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [54]:
dfs.keys()

dict_keys(['MaunaLoaMonthlyCO2', 'euromed2k', 'MaunaLoaWeeklyHistoricalCO2', 'geologictimescale', 'germany_june_meantemp', 'sunspots11000yrs', 'germany_august_meantemp', 'germany_july_meantemp', 'sunspotsmonthlymean1749', 'gisp2icecoretemp', 'gisp2icecoreaccum', 'CenozoicCO2', 'CenozoicCO2Filtered'])

In [ ]:
for dataset_name in dfs.keys():
    print()
    print("---------------------------------------------------------------------------------")
    print(f"Dataset {dataset_name} {dfs[dataset_name]['metadata']['tags']}")
    print()
    for entry in dfs[dataset_name]['metadata']:
        if entry in ['title', 'tags']:
            continue
        display(HTML(f"{entry}: {dfs[dataset_name]['metadata'][entry]}"))
    display(dfs[dataset_name]['data'])

In [ ]:
em2k=dfs['euromed2k']['data'].to_numpy()
gjmt6=dfs['germany_june_meantemp']['data'].to_numpy()
gjmt7=dfs['germany_july_meantemp']['data'].to_numpy()
gjmt8=dfs['germany_august_meantemp']['data'].to_numpy()
sunsp=dfs['sunspots11000yrs']['data'].to_numpy()
sunsp1749=dfs['sunspotsmonthlymean1749']['data'].to_numpy()

In [ ]:
sunsp[:,0]=1950-sunsp[:,0]

In [ ]:
gjmt=(gjmt6[:-1,:]+gjmt7+gjmt8)/3.0

In [ ]:
np.mean(gjmt[:,2])

In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
gjmean=np.mean(gjmt[:,2])

In [ ]:
win=10
em2kmat=moving_average(em2k[:,1],2*win+1)
gjmtmat=moving_average(gjmt[:,2]-gjmean,2*win+1)

In [ ]:
sunma=moving_average(sunsp[:,1],2*win+1)
sunman=moving_average(sunsp1749[:,3],20*win+1)

In [ ]:
sunman

In [ ]:
sunman.shape, sunsp1749.shape

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14,8]  # inch! :-/

In [ ]:
plt.plot(gjmt[:,0],gjmt[:,2]-gjmean)
plt.plot(em2k[:,0],em2k[:,1])
plt.plot(sunsp[-200:,0],sunsp[-200:,1]/60-0.4)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200)

In [ ]:
# plt.plot(dfnp[:,0],dfnp[:,1])
plt.plot(gjmt[win:-win,0],gjmtmat)
plt.plot(em2k[win:-win,0],em2kmat)
plt.plot(sunsp[-200:,0],sunsp[-200:,1]/80-0.5) # sunma[-200:]/50-0.5)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200-0.5)

In [ ]:
plt.plot(em2k[-120+win:-win,0],em2kmat[-120+win:-win])
plt.plot(gjmt[win:-win,0],gjmtmat)
plt.plot(sunsp[-20:,0],sunsp[-20:,1]/80-0.5) # sunma[-20:]/50-0.5)
plt.plot(sunsp1749[10*win:-10*win,2],sunman/200-0.5)

In [ ]:
a=-1500
b=-1400
plt.plot(em2k[a:b,0],em2k[a:b,1])
plt.plot(gjmt[a:b,0],gjmtmat[a-win:b-win])
